In [6]:
%load_ext autoreload
%autoreload 2

from config import imap_password_customplanet, db_username, db_password
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import uuid
from modules.db_operations import DatabaseConnector
from modules.sends import SendMail
EMAIL_ADDRESS_FROM = 'team@customplanet.com'
EMAIL_PASS = imap_password_customplanet


pd.set_option('display.max_columns', None)
#This can be further cleansed in the future
df = pd.read_csv('KC SCHOOLS - KC SCHOOLS.csv')


#Temporary testing to modify the Athletic Director email to my own
df.loc[0, 'Athletic Director.1'] = '2015samtaylor@gmail.com'
df.loc[1, 'Athletic Director.1'] = 'sammytaylor2006@yahoo.com'
df.loc[2, 'Athletic Director.1'] = 'samuel.taylor@greendot.org'
df.loc[3, 'Athletic Director.1'] = 'sammytaylor2006@gmail.com'
df.loc[3, 'Athletic Director.1'] = 'jerrybons2006@gmail.com'


def process(contact_column, sport, email_campaign_name, global_subject_line):

    data_list = []
    processed_emails = set()

    for index, row in df[:1].iterrows():

        # Get the current date
 
# Set the time zone to Central Time
        central_time_zone = pytz.timezone('America/Chicago')
        now_central = datetime.now(central_time_zone)
        formatted_date = now_central.strftime("%m%d%Y%H%M%S")
      
        school = row['School Name']
        png_path = row['PNG_PATH']
        contact = row[contact_column]
         #estiablish unique identifier, in order to match up FK relation, and merges
        unique_identifier = str(uuid.uuid4())

        # Check if the email has already been processed
        if contact in processed_emails:
            print(f"Skipping email to {contact} as it has already been processed.")
            continue

        data = [school, contact, sport, formatted_date, unique_identifier]
        data_list.append(data)

        # Mark the email as processed
        processed_emails.add(contact)
        
        SendMail.send(EMAIL_ADDRESS_FROM, EMAIL_PASS, global_subject_line, school, sport, png_path, contact, unique_identifier)

    
    data_list = pd.DataFrame(data_list, columns=['School', 'Contact', 'Sport', 'Date_Sent', 'recipient_id'])

    data_list.rename(columns = {'Contact': 'contact_email', 'School': 'school', 'Date_Sent': 'date_sent', 'Sport': 'sport'}, inplace = True)
    data_list['date_sent'] = data_list['date_sent'].astype(str)

    data_list['subject_line'] = global_subject_line
    data_list['contact_position'] = contact_column
    data_list['sent_from'] = EMAIL_ADDRESS_FROM
    data_list['email_campaign_tag'] = email_campaign_name
    data_list['date_sent'] = now_central.strftime("%Y-%m-%d %H:%M:%S")

    data_list.to_csv('output.csv', index=False)

    return(data_list)

#Subject line references format string in sends.py
global global_subject_line
global_subject_line = 'Local Fall {} Jerseys for {} 3'


# #Must be the column name of the position wanting to send, sport fills in email headers, third arg is campaign name
email_history = process('Athletic Director.1', 'Basketball', 'Initial', global_subject_line)

# Specify database connection details
server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
database = 'emailcampaign'
db = 'emailcampaign'
table_name = 'email_history'

# Instantiate the DatabaseConnector class, and send over the dataframe via append
db_connector = DatabaseConnector(server, database, db_username, db_password, db, table_name)
db_connector.send(email_history)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Sending_Emails\modules\db_operations.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)


In [3]:
e = '<div dir="ltr">*Test for scraping responses, and NLP<div><br></div><div>Hello,\xa0</div><div><br></div><div>I am very interested in checking out these basketball jerseys, the timeline which we need them by is December 25th. Is this something that can be done?</div><div>Also I would like help with my design. Can you help me out with these things?</div><div><br></div><div>Thanks</div></div><br><div class="gmail_quote"><div dir="ltr" class="gmail_attr">On Thu, Nov 30, 2023 at 7:49\u202fPM &lt;<a href="mailto:team@customplanet.com">team@customplanet.com</a>&gt; wrote:<br></div><blockquote class="gmail_quote" style="margin:0px 0px 0px 0.8ex;border-left:1px solid rgb(204,204,204);padding-left:1ex"><div class="msg7027903781100203139">\r\n            <u></u>\r\n            \r\n            \r\n                \r\n            \r\n            <div>\r\n                <div class="m_7027903781100203139container">\r\n                    <h1>Hello F.L. Schlagle High School (KCKS)]!</h1>\r\n                    <p>We are excited to introduce ourselves as your local supplier for <span class="m_7027903781100203139underline">Custom Sports Jerseys and apparel</span>. Elevate your team&#39;s spirit with our high-quality, personalized jerseys designed to make a statement on the field.</p>\r\n                    <p>Why choose us?</p>\r\n                    <ul>\r\n                        <li>Custom Designs: Bring your unique vision to life with our customizable design options.</li>\r\n                        <li>Quality Materials: Our jerseys are crafted from premium materials for comfort and durability.</li>\r\n                        <li>Quick Turnaround: Fast and efficient service to meet your tight schedules.</li>\r\n                    </ul>\r\n                    <p>Ready to get started?</p>\r\n                    <p>Visit our website or give us a call to request a free sample and explore our catalog: <a href="http://customplanet.com" class="m_7027903781100203139underline" target="_blank">customplanet.com</a>.</p>\r\n                    \r\n                     <div class="m_7027903781100203139image-container">\r\n                        <img src="https://lh3.googleusercontent.com/p/AF1QipMOx-lGQC-IlQwU0kBo5uP18BkuLrDoKAKGDS9h=s1360-w1360-h1020" alt="Customplanet Culture 1" width="300px">\r\n                        <img src="https://lh3.googleusercontent.com/p/AF1QipNLF0i0wl_6Y4VF9vHJKpjGtMHhgT70_ADLww8j=s1360-w1360-h1020" alt="Customplanet Culture 2" width="300px">\r\n                    </div>\r\n                    <p>Best Regards F.L. Schlagle High School (KCKS),<br>Your Team at Customplanet</p>\r\n                    <p>Basketball season is nearly here!</p>\r\n                    <p>For inquiries, contact us at: <a class="m_7027903781100203139underline"> (801) 810-8337</a></p>\r\n                </div>\r\n            </div>\r\n            \r\n        \r\n</div></blockquote></div>\r\n' 
from pprint import pprint
pprint(e)

('<div dir="ltr">*Test for scraping responses, and '
 'NLP<div><br></div><div>Hello,\xa0</div><div><br></div><div>I am very '
 'interested in checking out these basketball jerseys, the timeline which we '
 'need them by is December 25th. Is this something that can be '
 'done?</div><div>Also I would like help with my design. Can you help me out '
 'with these things?</div><div><br></div><div>Thanks</div></div><br><div '
 'class="gmail_quote"><div dir="ltr" class="gmail_attr">On Thu, Nov 30, 2023 '
 'at 7:49\u202fPM &lt;<a '
 'href="mailto:team@customplanet.com">team@customplanet.com</a>&gt; '
 'wrote:<br></div><blockquote class="gmail_quote" style="margin:0px 0px 0px '
 '0.8ex;border-left:1px solid rgb(204,204,204);padding-left:1ex"><div '
 'class="msg7027903781100203139">\r\n'
 '            <u></u>\r\n'
 '            \r\n'
 '            \r\n'
 '                \r\n'
 '            \r\n'
 '            <div>\r\n'
 '                <div class="m_7027903781100203139container">\r\n'
 '     

# To DO

In [ ]:
#Put the PNG on the given sport jersey
#Clean the CSV to iterate properly, this should be done. 

#Singularize the db_operations to reference in both

#The image can be embedded as an attachment. 
# When in the HTML is seems that is must be online, and not local. 

In [ ]:
import pandas as pd

df = pd.read_csv(r'C:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Sending_Emails\KC SCHOOLS - KC SCHOOLS.csv')